# Download the data, combine with background and binning example (4C+21.35)

**This tutotrial walks through all the steps needed in creating the combined data set (source + background), and binning the data for the blazar (4C+21.35, noflare).**

**For the background we use just the albedo photons.**

**To run this, you need the following files (available on wasabi)**

- COSI-SMEX/DC3/Data/Sources/4C21p35_noflare_3months_unbinned_data_filtered_with_SAAcut.fits.gz 
- COSI-SMEX/DC3/Data/Backgrounds/Ge/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz  
 

In [1]:
from cosipy import COSILike, BinnedData
from cosipy.spacecraftfile import SpacecraftFile
from cosipy.response.FullDetectorResponse import FullDetectorResponse
from cosipy.util import fetch_wasabi_file

from scoords import SpacecraftFrame

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.stats import poisson_conf_interval

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from threeML import Band, PointSource, Model, JointLikelihood, DataList
from cosipy import Band_Eflux
from astromodels import Parameter

from pathlib import Path

import os
import corner
from threeML import*


07:45:02 WARNING   The naima package is not available. Models that depend on it will not be         ]8;id=388395;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=400477;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py#47\47]8;;\
                  available                                                                                        

         WARNING   The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it  ]8;id=807184;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=316093;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py#68\68]8;;\
                  will not be available.                                                                           

         WARNING   The ebltable package is not available. Models that depend on it will not be     ]8;id=442046;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/absorption.py\absorption.py]8;;\:]8;id=101500;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/absorption.py#33\33]8;;\
                  available                                                                                        

         INFO      Starting 3ML!                                                                     ]8;id=381518;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=33879;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#39\39]8;;\

         WARNING   WARNINGs here are NOT errors                                                      ]8;id=987519;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=755632;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#40\40]8;;\

...

         WARNING   Could not import plugin HAWCLike.py. Do you have the relative instrument         ]8;id=841772;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=479765;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#144\144]8;;\
                  software installed and configured?                                                               

         WARNING   No fermitools installed                                              ]8;id=92777;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py\lat_transient_builder.py]8;;\:]8;id=393122;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py#44\44]8;;\

         WARNING   Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=587063;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=184780;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=878351;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=904114;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal     ]8;id=458624;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=652904;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

## Get the data

Define the path to the directory to save the data. The data can be downloaded by running the cells below. Each respective cell also gives the wasabi file path and file size.

In [2]:
data_path = Path("/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset")


Download the source file (9.7 MB)

In [3]:
# 4C+21.35 source
# File size: 9.7 MB

fetch_wasabi_file('COSI-SMEX/DC3/Data/Sources/4C21p35_noflare_3months_unbinned_data_filtered_with_SAAcut.fits.gz', output=str(data_path / '4C21p35_noflare_3months_unbinned_data_filtered_with_SAAcut.fits.gz'))


{
    "AcceptRanges": "bytes",
    "LastModified": "Thu, 13 Feb 2025 17:00:49 GMT",
    "ContentLength": 3384313,
    "ETag": "\"ee7135795540824dce6b1df8e7fe43d2\"",
    "ContentType": "application/gzip",
    "Metadata": {}
}


Download the background file (2.7 GB)

In [4]:
# albedo photons
# File size: 2.7 GB
fetch_wasabi_file('COSI-SMEX/DC3/Data/Backgrounds/Ge/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz', output=str(data_path / 'AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz'))


{
    "AcceptRanges": "bytes",
    "LastModified": "Mon, 17 Feb 2025 14:26:03 GMT",
    "ContentLength": 2903157095,
    "ETag": "\"80f088590996182860c195d0a383a6c9-29\"",
    "ContentType": "application/gzip",
    "Metadata": {}
}


Create the combined data

We will combine the 4C+21.35 source and the albedo photon background, which will be used as our dataset. This only needs to be done once. You can skip this cell if you already have the combined data file.


In [5]:
# Define instance of binned data class:
instance = BinnedData("blazar.yaml")

# Combine files:

input_files = ["/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz","/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/4C21p35_noflare_3months_unbinned_data_filtered_with_SAAcut.fits.gz"]
instance.combine_unbinned_data(input_files, output_name="4C21p35_noflare_AlbedoPhoton_bkg")


Bin the data

You only have to do this once, and after you can start by loading the binned data directly. You can skip this cell if you already have the binned data files.

In [6]:
# Bin 4C+21.35:
flare = BinnedData("blazar.yaml")

flare.get_binned_data(unbinned_data="/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/4C21p35_noflare_3months_unbinned_data_filtered_with_SAAcut.fits.gz", output_name="4C21p35_noflare_binned_data")


In [7]:
# Bin background:
bg_tot = BinnedData("blazar.yaml")

bg_tot.get_binned_data(unbinned_data="/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz", output_name="AlbedoPhoton_bkg_binned_data")


In [8]:
# Bin combined data:
data_combined = BinnedData("blazar.yaml")

data_combined.get_binned_data(unbinned_data="/home/srinadb/cosipy/docs/tutorials/spectral_fits/continuum_fit/blazar/4C21p35_noflare_AlbedoPhoton_bkg.fits.gz", output_name="4C21p35_noflare_AlbedoPhoton_bkg_binned_data")


### No Flare

In [9]:
# https://github.com/cositools/cosi-sim/blob/main/cosi_sim/Source_Library/DC3/sources/Extragalactic/4C21p35_noflare/4C21p35_noflare.source

total_integrated_flux = 0.0005535336768481181 # ph/cm^2/s 0.0005535336768481181
alpha = -1.6

# F_int = (K / (E_piv^alpha * (alpha+1))) * [ E_max^(alpha+1) - E_min^(alpha+1) ]
# K = F_int * (E_piv^alpha * (alpha+1)) / [ E_max^(alpha+1) - E_min^(alpha+1) ]

E_min = 100.0
E_max = 10000.0
E_piv = 1000000

n = total_integrated_flux * (E_piv**alpha) * (1+alpha)
d = (E_max**(1+alpha) - E_min**(1+alpha))

k = n/d
print(k)

1.4112374166612337e-12


In [13]:
# https://github.com/cositools/cosi-sim/blob/main/cosi_sim/Source_Library/DC3/sources/Extragalactic/4C21p35_noflare/4C_21_spectrum_noflare.dat

import math 

# --- Configuration ---
FILENAME = './4C_21_spectrum_noflare.dat'  # Name of your data file
ALPHA = -1.6                    # The alpha parameter from the formula
E_PIV_GEV = 1.0                # Pivot energy in GeV

# --- Constants and Conversions ---
E_PIV_KEV = E_PIV_GEV * 1e6    # Convert Pivot Energy to keV
EXPONENT = ALPHA              # The actual exponent in the formula dN/dE = K*(E/E_piv)^(-alpha)

# --- Function to calculate K for a single point ---
def calculate_single_k(energy_kev, dNdE, e_piv_kev, exponent):
    """
    Calculates K for a single energy point using dN/dE = K * (E/E_piv)^exponent.
    Returns K or None if calculation fails.
    """
    if energy_kev <= 0 or e_piv_kev <= 0 or dNdE < 0:
        print(f"Warning: Invalid input for K calculation "
              f"(E={energy_kev:.2e}, dN/dE={dNdE:.2e}). Skipping.")
        return None

    try:
        # Formula: K = dNdE / ( (E / E_piv)^exponent )
        base = energy_kev / e_piv_kev
        denominator = math.pow(base, exponent)

        if denominator == 0:
            print(f"Warning: Denominator is zero for E={energy_kev:.2e}. Skipping K calculation.")
            return None

        K = dNdE / denominator
        return K

    except (OverflowError, ValueError) as e:
        print(f"Error calculating K for E={energy_kev:.2e}: {e}. Skipping.")
        return None
    except ZeroDivisionError:
        print(f"Warning: Division by zero (base likely zero) for E={energy_kev:.2e}. Skipping K calculation.")
        return None


# --- Main Script Logic ---
k_values = []
energies = []
dNdE_values = []

print(f"Reading data from: {FILENAME}")
print(f"Using parameters: alpha={ALPHA}, E_piv={E_PIV_GEV} GeV ({E_PIV_KEV:.1e} keV)")
print(f"Using formula: dN/dE = K * (E / E_piv)^{EXPONENT:.2f}")
print("-" * 30)

# Check if file exists
if not os.path.exists(FILENAME):
    print(f"Error: File not found at '{FILENAME}'")
    exit() # Stop execution if file doesn't exist

# Read the file and process data lines
try:
    with open(FILENAME, 'r') as f:
        for i, line in enumerate(f):
            line = line.strip() # Remove leading/trailing whitespace
            if line.startswith('DP '):
                parts = line.split()
                if len(parts) >= 3:
                    try:
                        energy_kev = float(parts[1])
                        spectrum_dNdE = float(parts[2]) # This is dN/dE

                        # Calculate K for this point
                        k_single = calculate_single_k(energy_kev, spectrum_dNdE, E_PIV_KEV, EXPONENT)

                        # Store values if K calculation was successful
                        if k_single is not None:
                            k_values.append(k_single)
                            energies.append(energy_kev)
                            dNdE_values.append(spectrum_dNdE)
                            # Optional: print K for each point as it's calculated
                            # print(f"  Line {i+1}: E={energy_kev:.3e}, dN/dE={spectrum_dNdE:.3e} -> K={k_single:.3e}")

                    except ValueError:
                        print(f"Warning: Could not parse numbers on line {i+1}: '{line}'. Skipping.")
                    except IndexError:
                         print(f"Warning: Not enough columns on line {i+1}: '{line}'. Skipping.")
                else:
                    print(f"Warning: Malformed 'DP' line {i+1}: '{line}'. Skipping.")

except IOError as e:
    print(f"Error reading file '{FILENAME}': {e}")
    exit()

print("-" * 30)

# --- Calculate and Print Average K ---
if k_values: # Check if the list is not empty
    average_k = sum(k_values) / len(k_values)
    print(f"Successfully calculated K for {len(k_values)} data points.")
    print(f"Average K value: {average_k:.9e} ph cm^-2 s^-1 keV^-1")

    # Optional: Print a summary table
    print("\nIndividual Point Summary:")
    print("Energy (keV) | dN/dE        | Calculated K")
    print("----------------------------------------------")
    for e, dn, k in zip(energies, dNdE_values, k_values):
        print(f"{e:<12.5e} | {dn:<12.5e} | {k:<12.5e}")

else:
    print("No valid K values were calculated from the file.")




Reading data from: ./4C_21_spectrum_noflare.dat
Using parameters: alpha=-1.6, E_piv=1.0 GeV (1.0e+06 keV)
Using formula: dN/dE = K * (E / E_piv)^-1.60
------------------------------
------------------------------
Successfully calculated K for 493 data points.
Average K value: 1.450068733e-12 ph cm^-2 s^-1 keV^-1

Individual Point Summary:
Energy (keV) | dN/dE        | Calculated K
----------------------------------------------
1.00000e+02  | 3.60000e-06  | 1.43319e-12 
1.08000e+02  | 3.23700e-06  | 1.45754e-12 
1.09000e+02  | 3.18900e-06  | 1.45726e-12 
1.10000e+02  | 3.14200e-06  | 1.45692e-12 
1.11000e+02  | 3.09600e-06  | 1.45652e-12 
1.12000e+02  | 3.05100e-06  | 1.45610e-12 
1.13000e+02  | 3.00600e-06  | 1.45517e-12 
1.14000e+02  | 2.96200e-06  | 1.45423e-12 
1.15000e+02  | 2.91900e-06  | 1.45328e-12 
1.16000e+02  | 2.87600e-06  | 1.45185e-12 
1.17000e+02  | 2.83400e-06  | 1.45043e-12 
1.18000e+02  | 2.79200e-06  | 1.44853e-12 
1.19000e+02  | 2.75100e-06  | 1.44666e-12 
1.20000e+0